# Chemical Reactions: The Reacter Module

Imagine if you could **program** chemical reactions like building with LEGO blocks 🧱

That's exactly what `molpy.reacter` lets you do! ✨

## Why Reacter?

In molecular simulation, we frequently need to:

- 🧪 Build polymers
- 🔗 Simulate chemical reactions
- 🧬 Generate complex molecular structures

Traditional approaches either rely on external tools (like RDKit's SMIRKS) or require writing complex atom manipulation code from scratch.

**Reacter's Design Philosophy:**

- **Pure Python** - No RDKit dependency, fully native molpy data structures
- **Composable** - Reaction logic = modular function composition
- **Auditable** - All changes recorded in `ProductSet.notes`
- **Stable Indexing** - Atom deletion doesn't scramble IDs

---

## Core Concept: Reactions as Recipes

Think of a `Reacter` as a **chemical recipe** composed of three key ingredients:

```
┌─────────────────────────────────────────────┐
│           Reacter (Reaction)                │
│                                             │
│  1️⃣  Selectors                              │
│      ├─ Find reaction sites (anchors)       │
│      └─ Find atoms to remove (leaving grps) │
│                                             │
│  2️⃣  Transformers                           │
│      └─ Create new chemical bonds           │
│                                             │
│  3️⃣  ProductSet                             │
│      ├─ Reaction product                    │
│      └─ Metadata (removed atoms, new bonds) │
└─────────────────────────────────────────────┘
```

Let's dive into a real chemical reaction to understand these concepts!

---

## Your First Reaction: Esterification ⭐

We'll simulate a classic organic chemistry reaction: **acetic acid + ethanol → ethyl acetate**

```
CH₃COOH  +  CH₃CH₂OH  →  CH₃COOCH₂CH₃  +  H₂O
(acetic acid) (ethanol)   (ethyl acetate)  (water)
```

The reaction mechanism:
- Acetic acid loses an **-OH** group
- Ethanol loses an **-H**
- They connect via a **C-O** ester bond
- The leaving OH and H combine to form water

### Step 1: Build the Reactants

First, let's construct the molecular structures of acetic acid and ethanol:

In [16]:
from molpy import Atom, Bond
from molpy.core.wrappers.monomer import Monomer


def create_acetic_acid():
    """Create acetic acid: CH3-COOH"""
    mono = Monomer(name="acetic_acid")

    # CH3 methyl group
    c1 = Atom(symbol="C", name="C1")
    h1 = Atom(symbol="H", name="H1")
    h2 = Atom(symbol="H", name="H2")
    h3 = Atom(symbol="H", name="H3")

    # COOH carboxyl group
    c2 = Atom(symbol="C", name="C2")  # Carbonyl carbon
    o1 = Atom(symbol="O", name="O1")  # C=O double-bonded oxygen
    o2 = Atom(symbol="O", name="O2")  # C-OH hydroxyl oxygen
    h4 = Atom(symbol="H", name="H4")  # Hydroxyl hydrogen

    mono.add_entity(c1, h1, h2, h3, c2, o1, o2, h4)

    # Bonds
    mono.add_link(
        Bond(c1, h1, order=1),
        Bond(c1, h2, order=1),
        Bond(c1, h3, order=1),
        Bond(c1, c2, order=1),  # CH3-C
        Bond(c2, o1, order=2),  # C=O (carbonyl)
        Bond(c2, o2, order=1),  # C-OH (hydroxyl)
        Bond(o2, h4, order=1),  # O-H
    )

    # Set port: carbonyl carbon is the reaction site
    mono.set_port("1", c2)

    return mono


def create_ethanol():
    """Create ethanol: CH3-CH2-OH"""
    mono = Monomer(name="ethanol")

    # CH3 methyl group
    c1 = Atom(symbol="C", name="C1")
    h1 = Atom(symbol="H", name="H1")
    h2 = Atom(symbol="H", name="H2")
    h3 = Atom(symbol="H", name="H3")

    # CH2OH hydroxymethyl group
    c2 = Atom(symbol="C", name="C2")
    h4 = Atom(symbol="H", name="H4")
    h5 = Atom(symbol="H", name="H5")
    o = Atom(symbol="O", name="O")
    h6 = Atom(symbol="H", name="H6")  # This H will be removed

    mono.add_entity(c1, h1, h2, h3, c2, h4, h5, o, h6)

    # Bonds
    mono.add_link(
        Bond(c1, h1, order=1),
        Bond(c1, h2, order=1),
        Bond(c1, h3, order=1),
        Bond(c1, c2, order=1),  # CH3-CH2
        Bond(c2, h4, order=1),
        Bond(c2, h5, order=1),
        Bond(c2, o, order=1),  # CH2-O
        Bond(o, h6, order=1),  # O-H
    )

    # Set port: oxygen is the reaction site
    mono.set_port("1", o)

    return mono


# Create the reactants
acetic_acid = create_acetic_acid()
ethanol = create_ethanol()

print(f"Acetic acid: {len(list(acetic_acid.atoms))} atoms")
print(f"Ethanol: {len(list(ethanol.atoms))} atoms")

Acetic acid: 8 atoms
Ethanol: 9 atoms


### Step 2: Define the Esterification Reaction

Now we assemble the "reaction recipe":

In [17]:
from molpy.reacter import (
    Reacter,
    select_port_atom,  # Find reaction sites via ports
    select_hydroxyl_group,  # Remove -OH group
    select_one_hydrogen,  # Remove one hydrogen
    form_single_bond,  # Create single bond
)

# Create esterification reacter
esterification = Reacter(
    name="esterification",
    # Left reactant: acetic acid
    port_selector_left=select_port_atom,  # Find carbonyl carbon via port
    leaving_selector_left=select_hydroxyl_group,  # Remove -OH group
    # Right reactant: ethanol
    port_selector_right=select_port_atom,  # Find oxygen via port
    leaving_selector_right=select_one_hydrogen,  # Remove -H
    # Bond formation
    bond_former=form_single_bond,  # Form C-O ester bond
)

print(f"Reacter: {esterification}")
print("Reaction mechanism:")
print("  Acetic acid -OH  →  (removed)")
print("  Ethanol -H       →  (removed)")
print("  C + O            →  C-O ester bond")

Reacter: Reacter(name='esterification')
Reaction mechanism:
  Acetic acid -OH  →  (removed)
  Ethanol -H       →  (removed)
  C + O            →  C-O ester bond


### Step 3: Run the Reaction!

Executing the reaction is as simple as pressing "start":

In [18]:
# Execute the reaction
product = esterification.run(
    acetic_acid,
    ethanol,
    port_L="1",  # Acetic acid port
    port_R="1",  # Ethanol port
)

print("=" * 50)
print("Reaction Complete!")
print("=" * 50)

# Inspect the product
print(f"\nProduct atoms: {len(list(product.product.atoms))}")
print(f"Removed atoms: {product.notes['n_eliminated']}")
print(f"New bonds: {len(product.notes['formed_bonds'])}")

# Calculate molecular formula
atom_counts = {}
for atom in product.product.atoms:
    symbol = atom.get("symbol")
    atom_counts[symbol] = atom_counts.get(symbol, 0) + 1

formula = "".join(
    f"{symbol}{atom_counts[symbol] if atom_counts[symbol] > 1 else ''}"
    for symbol in sorted(atom_counts.keys())
    if symbol
)

print(f"\nProduct formula: {formula}")
print("Expected formula: C4H8O2 (ethyl acetate)")

if formula == "C4H8O2":
    print("\n🎉 Success! Ethyl acetate formed!")
else:
    print(f"\n⚠️  Formula mismatch, check reaction")

Reaction Complete!

Product atoms: 14
Removed atoms: 3
New bonds: 1

Product formula: C4H8O2
Expected formula: C4H8O2 (ethyl acetate)

🎉 Success! Ethyl acetate formed!


### Understanding the ReactionProduct

`ReactionProduct` contains not just the product, but a complete "reaction log":

In [19]:
print("Reaction Details:")
print(f"  Reaction name: {product.notes['reaction_name']}")
print(f"  Left port: {product.notes['port_name_L']}")
print(f"  Right port: {product.notes['port_name_R']}")
print(f"  Removed atoms: {product.notes['n_eliminated']}")
print(f"  New bonds: {len(product.notes['formed_bonds'])}")
print(f"  Needs retypification: {product.notes['requires_retype']}")

print("\n💡 Tip:")
print("  - eliminated_atoms contains the removed atom entities")
print("  - port_L/port_R records the reaction sites")
print("  - This metadata is invaluable for debugging!")

Reaction Details:
  Reaction name: esterification
  Left port: 1
  Right port: 1
  Removed atoms: 3
  New bonds: 1
  Needs retypification: True

💡 Tip:
  - eliminated_atoms contains the removed atom entities
  - port_L/port_R records the reaction sites
  - This metadata is invaluable for debugging!


---

## Selector Catalog: Finding the Right Atoms

Selectors handle two jobs:
1. **Find anchors** - where the reaction occurs
2. **Find leaving groups** - which atoms to remove

### Port Selectors

#### `select_port_atom` - The Standard

The most common selector, finds reaction sites via ports:

In [20]:
from molpy.reacter import select_port_atom

# We already set ports in our example:
# acetic_acid.ports.add("1", c2)  # Carbonyl carbon
# ethanol.ports.add("1", o)        # Oxygen

# select_port_atom automatically finds these atoms
anchor = select_port_atom(acetic_acid, "1")
print(f"Acetic acid reaction site: {anchor.get('name')} ({anchor.get('symbol')})")

anchor = select_port_atom(ethanol, "1")
print(f"Ethanol reaction site: {anchor.get('name')} ({anchor.get('symbol')})")

Acetic acid reaction site: C2 (C)
Ethanol reaction site: O (O)


### Leaving Group Selectors

#### `select_one_hydrogen` - Remove One Hydrogen

In [21]:
from molpy.reacter import select_one_hydrogen

# Create a simple methane molecule CH4
test_mono = Monomer()
c = Atom(symbol="C")
h1 = Atom(symbol="H")
h2 = Atom(symbol="H")
h3 = Atom(symbol="H")
h4 = Atom(symbol="H")

test_mono.add_entity(c, h1, h2, h3, h4)
test_mono.add_link(Bond(c, h1), Bond(c, h2), Bond(c, h3), Bond(c, h4))

leaving = select_one_hydrogen(test_mono, c)
print(f"select_one_hydrogen: removes {len(leaving)} H atom(s)")
print("Use case: condensation reactions needing single H removal")

select_one_hydrogen: removes 1 H atom(s)
Use case: condensation reactions needing single H removal


#### `select_all_hydrogens` - Remove All Hydrogens

In [22]:
from molpy.reacter import select_all_hydrogens

leaving = select_all_hydrogens(test_mono, c)
print(f"select_all_hydrogens: removes {len(leaving)} H atoms")
print("Use case: forming unsaturated bonds (e.g., C=C double bonds)")

select_all_hydrogens: removes 4 H atoms
Use case: forming unsaturated bonds (e.g., C=C double bonds)


#### `select_hydroxyl_group` - Remove Hydroxyl Group

✨ **Now works correctly for carboxylic acids!** It distinguishes between C=O (double bond) and C-OH (single bond).

In [23]:
from molpy.reacter import select_hydroxyl_group

print("select_hydroxyl_group handles:")
print("  ✓ Alcohols (R-OH)")
print("  ✓ Phenols (Ar-OH)")
print("  ✓ Carboxylic acids (R-COOH) - distinguishes C=O from C-OH!")
print("\nHow it works:")
print("  - Checks bond order to oxygen neighbors")
print("  - Selects only single-bonded O (hydroxyl)")
print("  - Ignores double-bonded O (carbonyl)")

select_hydroxyl_group handles:
  ✓ Alcohols (R-OH)
  ✓ Phenols (Ar-OH)
  ✓ Carboxylic acids (R-COOH) - distinguishes C=O from C-OH!

How it works:
  - Checks bond order to oxygen neighbors
  - Selects only single-bonded O (hydroxyl)
  - Ignores double-bonded O (carbonyl)


#### `select_none` - Addition Reactions

In [24]:
from molpy.reacter import select_none

leaving = select_none(test_mono, c)
print(f"select_none: removes {len(leaving)} atoms")
print("Use case: addition reactions - no atoms removed, only bond formation")

select_none: removes 0 atoms
Use case: addition reactions - no atoms removed, only bond formation


---

## Transformer Catalog: Creating Bonds

Transformers create chemical bonds between anchor atoms.

### Basic Bond Types

In [25]:
from molpy.reacter import (
    form_single_bond,  # C-C  (order=1)
    form_double_bond,  # C=C  (order=2)
    form_triple_bond,  # C≡C  (order=3)
    form_aromatic_bond,  # C:C  (order=1.5)
)

print("Bond transformers:")
print("  form_single_bond   → C-C  (single, order=1)")
print("  form_double_bond   → C=C  (double, order=2)")
print("  form_triple_bond   → C≡C  (triple, order=3)")
print("  form_aromatic_bond → C:C  (aromatic, order=1.5)")

Bond transformers:
  form_single_bond   → C-C  (single, order=1)
  form_double_bond   → C=C  (double, order=2)
  form_triple_bond   → C≡C  (triple, order=3)
  form_aromatic_bond → C:C  (aromatic, order=1.5)


### Advanced: Factory Pattern

In [26]:
from molpy.reacter import create_bond_former

# Dynamically create bond formers with specific order
double_bond_maker = create_bond_former(2)
triple_bond_maker = create_bond_former(3)

print("Factory pattern: create transformers dynamically")
print("Use case: when bond order varies based on conditions")

Factory pattern: create transformers dynamically
Use case: when bond order varies based on conditions


### Special Transformers

In [27]:
from molpy.reacter import skip_bond_formation, break_bond

print("Special transformers:")
print("  skip_bond_formation → Don't create bonds (only remove leaving groups)")
print("  break_bond          → Break existing bonds")

Special transformers:
  skip_bond_formation → Don't create bonds (only remove leaving groups)
  break_bond          → Break existing bonds


---

## More Reaction Examples

### Example 1: C-C Coupling (Polymer Linkage)

In [28]:
# Create two simple C-H units
def create_ch_unit(name):
    mono = Monomer(name=name)
    c = Atom(symbol="C")
    h = Atom(symbol="H")
    mono.add_entity(c, h)
    mono.add_link(Bond(c, h))
    mono.set_port("1", c)
    return mono


mono_A = create_ch_unit("A")
mono_B = create_ch_unit("B")

# Create C-C coupling reaction
cc_coupling = Reacter(
    name="C-C_coupling",
    port_selector_left=select_port_atom,
    port_selector_right=select_port_atom,
    leaving_selector_left=select_one_hydrogen,
    leaving_selector_right=select_one_hydrogen,
    bond_former=form_single_bond,
)

product = cc_coupling.run(mono_A, mono_B, port_L="1", port_R="1")
print(f"C-C coupling: {product.notes['n_eliminated']} H atoms removed")
print(f"Product atoms: {len(list(product.product.atoms))} (2 carbons)")

C-C coupling: 2 H atoms removed
Product atoms: 2 (2 carbons)


### Example 2: Asymmetric Reaction (Forming Double Bonds)

In [29]:
# Create a carbon with multiple hydrogens
def create_ch2_unit():
    mono = Monomer()
    c = Atom(symbol="C")
    h1 = Atom(symbol="H")
    h2 = Atom(symbol="H")
    mono.add_entity(c, h1, h2)
    mono.add_link(Bond(c, h1), Bond(c, h2))
    mono.set_port("1", c)
    return mono


left = create_ch2_unit()
right = create_ch_unit("R")

# Asymmetric reaction: remove all H from left, one H from right, form double bond
double_bond_reaction = Reacter(
    name="form_double_bond",
    port_selector_left=select_port_atom,
    port_selector_right=select_port_atom,
    leaving_selector_left=select_all_hydrogens,  # Remove all H (2 atoms)
    leaving_selector_right=select_one_hydrogen,  # Remove one H
    bond_former=form_double_bond,  # Form C=C
)

product = double_bond_reaction.run(left, right, port_L="1", port_R="1")
print(f"Double bond formed: {product.notes['n_eliminated']} H atoms removed")
print(f"Bond order: {product.notes['formed_bonds'][0].get('order')} (double)")

Double bond formed: 3 H atoms removed
Bond order: 2 (double)


### Example 3: Addition Reaction (No Atom Removal)

In [30]:
def create_carbon():
    mono = Monomer()
    c = Atom(symbol="C")
    mono.add_entity(c)
    mono.set_port("1", c)
    return mono


c1 = create_carbon()
c2 = create_carbon()

# Pure addition: no atoms removed, only bond formation
addition = Reacter(
    name="addition",
    port_selector_left=select_port_atom,
    port_selector_right=select_port_atom,
    leaving_selector_left=select_none,
    leaving_selector_right=select_none,
    bond_former=form_single_bond,
)

product = addition.run(c1, c2, port_L="1", port_R="1")
print(f"Addition: {product.notes['n_eliminated']} atoms removed (should be 0)")
print(f"Product atoms: {len(list(product.product.atoms))} (2 carbons)")

Addition: 0 atoms removed (should be 0)
Product atoms: 2 (2 carbons)


---

## Advanced: Tracking Reaction Intermediates

Want to see each step of the reaction? Use `record_intermediates=True`:

In [31]:
# Re-run esterification with intermediate tracking
acetic_acid = create_acetic_acid()
ethanol = create_ethanol()

product = esterification.run(
    acetic_acid,
    ethanol,
    port_L="1",
    port_R="1",
    record_intermediates=True,  # Enable tracking!
)

# View reaction steps
if "intermediates" in product.notes:
    print("Reaction Steps:")
    for i, step in enumerate(product.notes["intermediates"], 1):
        print(f"\nStep {i}: {step['step']}")
        print(f"  {step['description']}")
        if "product" in step:
            atoms = len(list(step["product"].atoms))
            print(f"  Current atoms: {atoms}")

Reaction Steps:

Step 1: initial
  Initial reactants (Step 1-2: validated ports and port atoms)

Step 2: merge
  After merging right into left (Step 3)
  Current atoms: 17

Step 3: bond_formation
  After forming new bond between port atoms (Step 4)
  Current atoms: 17

Step 4: identify_leaving
  Identified leaving groups (Step 5a): 2 from left anchor, 1 from right anchor

Step 5: remove_leaving
  After removing 3 leaving atoms (Step 5b)
  Current atoms: 14

Step 6: topology
  Final topology computation (Step 6)
  Current atoms: 14


---

## Integration: Polymer Building

Reacter's most powerful use case is polymer construction. Via `ReacterConnector`, you can automate monomer linkage:

In [ ]:
from molpy.builder.polymer.linear import linear
from molpy.reacter import MonomerLinker

# Create monomer library
library = {
    "A": create_ch_unit("A"),
    "B": create_ch_unit("B"),
}

# Create a mapping for port connections
mapping = {
    ("A", "A"): ("1", "1"),
    ("A", "B"): ("1", "1"),
    ("B", "A"): ("1", "1"),
    ("B", "B"): ("1", "1"),
}

# Create connector using C-C coupling reaction
connector = MonomerLinker(
    mapping,
    default=cc_coupling  # Default to C-C coupling
)

# Build linear polymer
polymer = linear(
    sequence="ABABAB",  # Monomer sequence
    library=library,
    connector=connector,
)

print(f"Polymer built!")
print(f"Total atoms: {len(list(polymer.atoms))}")
print(f"Total bonds: {len(list(polymer.bonds))}")

# Tip: Use typifier for force field assignment
print("\n💡 Next step: Use OplsAtomisticTypifier for force field parameters")

TypeError: MonomerLinker.__init__() got multiple values for argument 'default_reaction'

---

## Best Practices & Debugging

### 1. Port Naming Conventions

```python
# ✓ Good naming
mono.set_port("head", carbon_1)  # Descriptive names
mono.set_port("tail", carbon_n)
mono.set_port("1", reactive_site)  # Numeric IDs

# ✗ Avoid
mono.set_port("x", atom)  # Unclear meaning
```

### 2. When to Retypify

After reactions, atom chemical environments change, requiring force field retypification:

In [ ]:
# Check if retypification needed
if product.notes["requires_retype"]:
    print("⚠️  Recommend force field retypification")
    print("   from molpy.typifier.atomistic import OplsAtomisticTypifier")
    print("   typifier = OplsAtomisticTypifier()")
    print("   typifier.typify(product.product)")

### 3. Debugging Reactions

If results don't match expectations:

In [ ]:
print("Debugging checklist:")
print("\n1. Verify ports:")
print("   port = mono.get_port('1')")
print("   print(port.target)  # Confirm targets correct atom")

print("\n2. Check leaving groups:")
print("   leaving = select_hydroxyl_group(mono, anchor)")
print("   print(f'Will remove {len(leaving)} atoms')")

print("\n3. Use record_intermediates=True to see each step")

print("\n4. Verify molecular formula:")
print("   Count atoms to confirm stoichiometry")

### 4. Performance Tips

For large-scale polymer building:

In [ ]:
print("Performance tips:")
print("  1. Avoid unnecessary record_intermediates (memory intensive)")
print("  2. Set compute_topology=False if angles/dihedrals not needed")
print("  3. For batch building, typify once at the end")
print("  4. For very large polymers (>10k atoms), consider chunked building")

---

## Summary

Congratulations! You've mastered Reacter's core concepts:

✅ **Reacter** = Selectors + Transformers  
✅ **Selectors** find reaction sites and leaving groups  
✅ **Transformers** create or break chemical bonds  
✅ **ReactionProduct** records the complete reaction history  
✅ **select_hydroxyl_group** now correctly handles carboxylic acids  
✅ **Integration** with builder enables automated polymer synthesis  

### Next Steps

- 📖 See `user-guide/builder.ipynb` for polymer building
- 📖 See `user-guide/typifier.ipynb` for force field assignment
- 🧪 Try implementing your own chemical reactions!

### References

- API docs: `docs/api/reacter.md`
- Test cases: `tests/test_reacter/`
- Source code: `src/molpy/reacter/`

Happy coding! 🎉


### When Things Go Wrong 🚫

What happens if a selector can't find the atoms it's looking for?

Reacter is designed to be **strict but safe**.
- If a port is missing -> `ValueError` (stops immediately)
- If a selector returns empty list -> Reaction proceeds (but nothing removed)

Let's see what happens if we try to remove an OH group from a molecule that doesn't have one (like Methane).


In [ ]:
# Create Methane (CH4)
methane = Monomer(name="methane")
c = Atom(symbol="C")
h = Atom(symbol="H")
methane.add_entity(c, h)
methane.add_link(Bond(c, h))

# Try to find OH group (spoiler: it won't find any)
leaving_groups = select_hydroxyl_group(methane, c)
print(f"Found leaving groups: {leaving_groups}")


### Custom Selectors: The Power of Python 🐍

Selectors are just Python functions! You can write your own logic to select any atom you want.

**Challenge:** Write a selector to remove any Halogen atom (F, Cl, Br, I).


In [ ]:
from molpy.reacter import find_neighbors

def remove_halogen(monomer, anchor):
    """Remove F, Cl, Br, or I bonded to anchor."""
    halogens = ["F", "Cl", "Br", "I"]
    neighbors = find_neighbors(monomer, anchor)
    # Return list of atoms that are halogens
    return [n for n in neighbors if n.get("symbol") in halogens]

# Test it on Chloromethane
chloromethane = Monomer(name="chloromethane")
c = Atom(symbol="C")
cl = Atom(symbol="Cl")
chloromethane.add_entity(c, cl)
chloromethane.add_link(Bond(c, cl))

leaving = remove_halogen(chloromethane, c)
if leaving:
    print(f"Found halogen to remove: {leaving[0].get('symbol')}")
else:
    print("No halogen found.")



## Another Example: Amide Formation 💊

Let's apply what we learned to a different reaction: **Amide Bond Formation**.
This is crucial for building proteins (peptide bonds) and materials like Nylon.

Reaction: **Carboxylic Acid + Amine → Amide + Water**


In [ ]:
# We can reuse our acid creator from before
acid = create_acetic_acid()

# Create Methylamine (CH3-NH2)
amine = Monomer(name="methylamine")
c = Atom(symbol="C")
n = Atom(symbol="N")
h1 = Atom(symbol="H")
h2 = Atom(symbol="H")
amine.add_entity(c, n, h1, h2)
amine.add_link(Bond(c, n), Bond(n, h1), Bond(n, h2))
amine.ports.add("1", n) # Anchor is Nitrogen

# Define Reaction
amide_reaction = Reacter(
    name="amide_formation",
    port_selector_left=select_port_atom,
    port_selector_right=select_port_atom,
    leaving_selector_left=select_hydroxyl_group,      # Acid loses OH
    leaving_selector_right=select_one_hydrogen,  # Amine loses H
    bond_former=form_single_bond, # Form C-N bond
)

# Run it!
product_amide = amide_reaction.run(acid, amine, port_L="1", port_R="1")
print(f"Created: {product_amide.notes['reaction_name']}")
print(f"New bonds: {len(product_amide.notes['formed_bonds'])}")


### Atomic Economy Check ⚖️

In simulations, mass conservation is key. Always verify that atoms aren't magically disappearing or appearing (except the ones you explicitly removed).

**Formula:** `Atoms In = Atoms Out + Atoms Removed`


In [ ]:
# Verify our Amide reaction
initial_atoms = len(list(acid.atoms)) + len(list(amine.atoms))
final_atoms = len(list(product_amide.product.atoms))
removed_atoms = product_amide.notes['n_eliminated']

print(f"Initial: {initial_atoms}")
print(f"Final:   {final_atoms}")
print(f"Removed: {removed_atoms}")

assert initial_atoms == final_atoms + removed_atoms
print("✅ Mass Conserved!")